## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-03-02-09-03-52 +0000,bbc,Oil prices jump after attack on Iran threatens...,https://www.bbc.com/news/articles/c75evve6l63o...
1,2026-03-02-09-02-37 +0000,nyt,"After Attack on Iran by the U.S. and Israel, S...",https://www.nytimes.com/2026/02/28/world/middl...
2,2026-03-02-09-01-46 +0000,nyt,Live Updates: War With Iran Escalates as Israe...,https://www.nytimes.com/live/2026/03/02/world/...
3,2026-03-02-09-01-31 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/03/02/world/...
4,2026-03-02-09-00-00 +0000,cbc,Deadline day: $20B submarine showdown puts Kor...,https://www.cbc.ca/news/politics/submarines-ca...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2592/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,iran,81
321,trump,22
19,strikes,21
99,iranian,18
9,israel,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2026-03-02-01-17-20 +0000,latimes,Thousands rally in Westwood as U.S.-Iran war e...,https://www.latimes.com/california/story/2026-...,189
225,2026-03-01-10-59-34 +0000,wapo,Iran’s supreme leader killed in U.S.-Israeli a...,https://www.washingtonpost.com/world/2026/02/2...,176
195,2026-03-01-15-31-00 +0000,nyt,Israel launches new military strikes as Iran r...,https://www.nytimes.com/live/2026/03/01/world/...,160
105,2026-03-01-22-24-27 +0000,bbc,Nine dead in missile attack on Israel as Iran ...,https://www.bbc.com/news/articles/c363zkp1pgxo...,149
1,2026-03-02-09-02-37 +0000,nyt,"After Attack on Iran by the U.S. and Israel, S...",https://www.nytimes.com/2026/02/28/world/middl...,147


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,189,2026-03-02-01-17-20 +0000,latimes,Thousands rally in Westwood as U.S.-Iran war e...,https://www.latimes.com/california/story/2026-...
225,95,2026-03-01-10-59-34 +0000,wapo,Iran’s supreme leader killed in U.S.-Israeli a...,https://www.washingtonpost.com/world/2026/02/2...
100,79,2026-03-01-22-57-46 +0000,nypost,Trump warns Iranian regime to surrender or fac...,https://nypost.com/2026/03/01/world-news/trump...
10,54,2026-03-02-08-31-00 +0000,wsj,Oil Prices Have Room to Rise Given Middle East...,https://www.wsj.com/finance/commodities-future...
74,41,2026-03-02-01-15-43 +0000,nypost,Family speaks out after FBI launched terror pr...,https://nypost.com/2026/03/01/us-news/oc-famil...
195,41,2026-03-01-15-31-00 +0000,nyt,Israel launches new military strikes as Iran r...,https://www.nytimes.com/live/2026/03/01/world/...
152,30,2026-03-01-19-32-00 +0000,wsj,Two Dead in Austin Bar Shooting; FBI Sees Poss...,https://www.wsj.com/us-news/two-dead-in-austin...
84,27,2026-03-01-23-55-08 +0000,latimes,WGA cancels Los Angeles awards show amid labor...,https://www.latimes.com/entertainment-arts/bus...
181,25,2026-03-01-16-35-43 +0000,nypost,Dems face mounting pressure to fund terror-fig...,https://nypost.com/2026/03/01/us-news/pressure...
54,24,2026-03-02-03-24-21 +0000,startribune,No. 22 Gophers women’s basketball clinches No....,https://www.startribune.com/no-22-gophers-wome...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
